In [29]:
## Init stuff
import maze
import mujoco
import mujoco.viewer as viewer
import numpy as np
import time
import matplotlib.pyplot as plt

## Joint lengths
kp = 0
kd = 0
l1 = 0.2
l2 = 0.5
l3 = 0.5

base_pos_x, base_pos_y, base_pos_z = 0, 0, 0
ball_pos_x, ball_pos_y, ball_pos_z = 0.2, -0.4, 0.2
maze_pos_x, maze_pos_y, maze_pos_z = -0.4, 0.8, 0

### TODO: look up tuning? dampratio="1" might already tune kv automatically
### but kp still needs to be tuned properly
### https://en.wikipedia.org/wiki/Ziegler%E2%80%93Nichols_method
kp1 = 200
kp2 = 200
kp3 = 200
kp4 = 1

grav = -9.81

robot_4dof = f'''
<mujoco model="3dof_arm">
    <compiler angle="radian" coordinate="local" meshdir="assets" autolimits="true"/>
    <option gravity="0 0 {grav}"/>

    <asset>
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
        rgb2=".2 .3 .4" width="300" height="300"/>
        <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
        <mesh name="maze" file="MazeIV.STL" scale="0.005 0.005 0.005"/>
        <mesh name="ee" file="ee1.STL" scale="0.05 0.05 0.05"/>
    </asset>
    
    <worldbody>
        <geom size="2 2 0.01" type="plane" material="grid"/>
        <light pos="0 0 1"/>
            
        <!-- Base -->
        <body name="base" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
            
            <!-- Joint 1 -->
            <body name="joint1" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
                <joint name="joint1_hinge" type="hinge" axis="0 0 1"/>
                <geom type="sphere" size="0.05" rgba="1 0 0 1" contype="1" conaffinity="0"/>
                
                <!-- Link 1 -->
                <geom type="cylinder" fromto="0 0 0 0 0 {l1}" size="0.02" rgba="0 1 0 1" contype="1" conaffinity="0"/>
                
                <!-- Joint 2 -->
                <body name="joint2" pos="0 0 {l1}" gravcomp="1">
                    <joint name="joint2_hinge" type="hinge" axis="0 1 0"/>
                    <geom type="sphere" size="0.05" rgba="1 0 0 1" contype="1" conaffinity="0"/>
                    
                    <!-- Link 2 -->
                    <geom type="cylinder" fromto="0 0 0 0 0 {l2}" size="0.02" rgba="0 1 0 1" contype="1" conaffinity="0"/>
                    
                    <!-- Joint 3 -->
                    <body name="joint3" pos="0 0 {l2}" gravcomp="1">
                        <joint name="joint3_hinge" type="hinge" axis="0 1 0"/>
                        <geom type="sphere" size="0.05" rgba="1 0 0 1" contype="1" conaffinity="0"/>
                        
                        <!-- Link 3 -->
                        <geom type="cylinder" fromto="0 0 0 0 0 {l3}" size="0.02" rgba="0 1 0 1" contype="1" conaffinity="0"/>
                        
                        <!-- End-effector -->
                        <body name="ee" pos="0 0 {l3}" gravcomp="1">
                            <joint name="joint4_hinge" type="hinge" axis="0 1 0"/>
                            <geom type="sphere" size="0.02" rgba="1 1 1 1" contype="1" conaffinity="0"/>
                            <geom type="mesh" mesh="ee" rgba="1 1 1 1" contype="1" conaffinity="0"/>
                            
                        </body>
                    </body>
                </body>
            </body>
        </body>

        <!-- Ball -->
        <body name="ball" pos="{ball_pos_x} {ball_pos_y} {ball_pos_z}">
            <geom type="sphere" size="0.03" rgba="1 1 0 1"/>
            <joint type="free"/>
        </body>

    </worldbody>

    <actuator>
        <position joint="joint1_hinge" kp="{kp1}" dampratio="1"/>
        <position joint="joint2_hinge" kp="{kp2}" dampratio="1"/>
        <position joint="joint3_hinge" kp="{kp3}" dampratio="1"/>
        <position joint="joint4_hinge" kp="{kp4}" dampratio="0.5"/>
    </actuator>
</mujoco>
'''

In [ ]:
## Simulation with joint positions controllable using GUI
## Use this to manually control the joints and do sanity checks

# Load model to reset robot
model = mujoco.MjModel.from_xml_string(robot_4dof)
data = mujoco.MjData(model)

with viewer.launch(model, data) as v:
    while v.is_running():
        mujoco.mj_step(model, data)
        v.render()

In [30]:
## IK

def shortest_angle_update(theta_current, theta_dest):
    # Finds closest angle configuration equivalent of goal configuration
    # to current configuration
    #  
    # theta_current: current angle measure
    # theta_dest: desired angle measure
    # returns the closest angle equivalent of theta_dest to theta_current
    diff = theta_dest - theta_current
    diff = (diff + np.pi) % (2*np.pi) - np.pi
    return theta_current + diff

def dist(point1, point2):
    # Finds distance between two points 
    # returns distance between the two points
    return np.linalg.norm(np.array(point2)[:2] - np.array(point1)[:2])

def forward_kinematics(theta1, theta2, theta3, l1, l2, l3):
    # Calculates the ee position given joint angles and arm lengths
    # Returns np.array([x, y, z]) of coordinates
    ##BUG: Returned position is nowhere near actual position
    s1 = np.sin(theta1)
    s2 = np.sin(theta2)
    s3 = np.sin(theta3)
    c1 = np.cos(theta1)
    c2 = np.cos(theta2)
    c3 = np.cos(theta3)
    x = -(l3*(c1*c2*c3 - c1*s2*s3) + c1*c2*l2)
    y = -(l3*(-s1*c2*c3 + s1*s2*s3) - s1*c2*l2)
    z = l3*(s2*c3 + c2*s3) + s2*l2 + l1

    return np.array([x, y, z])

def instructions_to_coordinates(instructions, start=np.array([-0.34, 0.31, 0.1]), step_size=0.11):
    # instructions: list of "U", "D", "L", "R" strings
    # step_size: distance moved for each instruction. needs tuning
    # returns a list of coordinates
    coordinates = [start]
    z_rise = 0.1 # keep the arm elevated
    for inst in instructions:
        x, y, z = coordinates[-1].copy()
        if inst == "U":
            y += step_size
        elif inst == "D":
            y -= step_size
        elif inst == "R":
            x += step_size
        elif inst == "L":
            x -= step_size
        else:
            raise ValueError(f"Invalid instruction: {inst}")
        coordinates.append(np.array([x, y, z_rise]))
    return coordinates


# Load model
model = mujoco.MjModel.from_xml_string(robot_4dof)
data = mujoco.MjData(model)

# Init parameters
ee_id = model.body('ee').id
base_id = model.body('base').id
ball_id = model.body('ball').id
base_xpos = data.body(base_id).xpos
tol = 0.1 # minimum distance from each step before moving on to the next (tune this)
update_freq = 1/60 # framerate/simulation rate. lower to speed up

# hard coded start of maze
start = np.array([-0.34, 0.13, 0.1])

# instructions = maze.start_end_to_instruction()
# instructions = ['R', 'R', 'R', 'R', 'R', 'U', 'L', 'U', 'U', 'R', 'U', 'L', 'L', 'L', 'U', 'R', 'R', 'R']
# coordinates = instructions_to_coordinates(instructions, start=start)
# coord_index = 0
t0 = time.time()

with mujoco.viewer.launch_passive(model, data) as v:
    while v.is_running():
        ##TODO: Maze traversal
        # goal = coordinates[coord_index]
        # ball_pos = data.body(ball_id).xpos
        # ee_pos = data.body(ee_id).xpos
        # error = dist(ee_pos, goal)
        # if error < tol:
        #     coord_index += 1
        #     if coord_index >= len(coordinates):
        #         coord_index = len(coordinates) - 1  # Remain at final point

        # draw circle
        t = time.time() - t0
        radius = 0.2
        y_center = 0.3
        x_center = 0
        z_center = 0.5
        omega = 1
        x_goal = radius * np.sin(omega * t) + x_center
        y_goal = y_center
        z_goal = radius * np.cos(omega * t) + z_center
        goal = np.array([x_goal, y_goal, z_goal])

        # Account for base position
        x_offset, y_offset, z_offset = np.subtract(goal, base_xpos)

        # Inverse kinematics
        ##BUG: general path looks decent, but all coordinates have a noticable error
        theta_curr = data.qpos.copy()
        theta1 = np.arctan2(-y_offset, -x_offset)
        r = np.sqrt(x_offset**2 + y_offset**2)
        theta3 = -np.arccos((x_offset**2 + y_offset**2 + z_offset**2 -l2**2-l3**2)/(2*l2*l3))
        theta2 = np.arcsin(z_offset/np.sqrt(x_offset**2 + y_offset**2 + z_offset**2)) + np.arctan2(l3*np.sin(theta3), l2+l3*np.cos(theta3))
        theta4 = np.pi - (theta2 + theta3)

        # Workaround for snapping between pi and -pi
        theta1_adj = shortest_angle_update(theta_curr[0], theta1)
        theta2_adj = shortest_angle_update(theta_curr[1], theta2)
        theta3_adj = shortest_angle_update(theta_curr[2], theta3)
        theta4_adj = np.pi-(theta2_adj + theta3_adj)

        # Sanity check and debugging
        ee_fk = forward_kinematics(theta1_adj, theta2_adj, theta3_adj, l1, l2, l3)
        print("Goal:\t", goal)
        print("Actual:\t", data.body(ee_id).xpos)
 
        # Send commands to the robot
        q = np.array([theta1_adj, theta2_adj, theta3_adj, theta4_adj])
        # q = np.array([0, 0, 0, 0])
        data.ctrl = q
        mujoco.mj_step(model, data)
        v.sync()
        time.sleep(update_freq)

Goal:	 [0.05558647 0.3        0.69212013]
Actual:	 [0.  0.  1.2]
Goal:	 [0.05899652 0.3        0.69110053]
Actual:	 [0.  0.  1.2]
Goal:	 [0.06243082 0.3        0.6900063 ]
Actual:	 [-0.00851246  0.00754886  1.19962171]
Goal:	 [0.06583832 0.3        0.68885263]
Actual:	 [-0.01740497  0.02294717  1.19751355]
Goal:	 [0.06909955 0.3        0.68768392]
Actual:	 [-0.01718824  0.03945418  1.19448592]
Goal:	 [0.07246811 0.3        0.68640916]
Actual:	 [-0.01464513  0.05536736  1.1902645 ]
Goal:	 [0.07581047 0.3        0.68507505]
Actual:	 [-0.01002929  0.07007657  1.1852633 ]
Goal:	 [0.07913788 0.3        0.68367688]
Actual:	 [-0.00424141  0.08426335  1.17942476]
Goal:	 [0.08242279 0.3        0.68222646]
Actual:	 [0.00229619 0.09827483 1.17276037]
Goal:	 [0.08569169 0.3        0.6807123 ]
Actual:	 [0.00935649 0.11201518 1.16542931]
Goal:	 [0.08881462 0.3        0.67919811]
Actual:	 [0.01674824 0.12531004 1.15760896]
Goal:	 [0.09202578 0.3        0.67757043]
Actual:	 [0.02429747 0.13804815 1.14